In [ ]:
# I want to calculate the NVDI Index of the different parts of the LA cities I am analyzing 
# I will calculate the NVDI for different time periods to see if there is a change in vegetation visible

In [2]:
!pip install rasterio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.8/18.8 MB 7.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [4]:
# Importing the necessary packages

import geopandas as gpd
import rasterio
from rasterio.mask import mask
import numpy as np
import matplotlib.pyplot as plt

In [18]:
# I will start with Mexico City

districts_mc = gpd.read_file("MexikoStadt_poligonos_colonias_cdmx")

In [12]:
districs_mc

,cve_ent,entidad,cve_alc,alc,cve_col,colonia,clasif,geometry
0,09,Ciudad de México,002,Azcapotzalco,002-001,Aguilera,Colonia,"POLYGON ((-99.15913 19.47261, -99.15906 19.473..."
1,09,Ciudad de México,002,Azcapotzalco,002-002,Aldana,Colonia,"POLYGON ((-99.15171 19.46819, -99.15157 19.469..."
2,09,Ciudad de México,002,Azcapotzalco,002-003,Ampliacion Cosmopolita,Colonia,"POLYGON ((-99.16362 19.47223, -99.16334 19.472..."
3,09,Ciudad de México,002,Azcapotzalco,002-004,Ampliacion Del Gas,Colonia,"POLYGON ((-99.16152 19.46774, -99.16152 19.468..."
4,09,Ciudad de México,002,Azcapotzalco,002-005,Ampliacion Petrolera,Colonia,"POLYGON ((-99.19762 19.48231, -99.19649 19.485..."
...,...,...,...,...,...,...,...,...
1538,09,Ciudad de México,017,Venustiano Carranza,017-063,Tres Mosqueteros,Colonia,"POLYGON ((-99.10707 19.43999, -99.10701 19.440..."
1539,09,Ciudad de México,017,Venustiano Carranza,017-064,Valentin Gomez Farias,Colonia,"POLYGON ((-99.09632 19.41984, -99.09597 19.419..."
1540,09,Ciudad de México,017,Venustiano Carranza,017-065,Valle Gomez,Colonia,"POLYGON ((-99.1237 19.4564, -99.12354 19.45706..."
1541,09,Ciudad de México,017,Venustiano Carranza,017-066,Venustiano Carranza,Colonia,"POLYGON ((-99.11429 19.43788, -99.11425 19.438..."


In [14]:
# Speichern der runtergeladenen Satellitenbilder
nir_mc = "T14QMG_20241226T170731_B8A_20m.jp2"  # NIR-Band (Sentinel-2 Band 8)
red_mc = "T14QMG_20241226T170731_B04_20m.jp2"  # Rot-Band (Sentinel-2 Band 4)

In [21]:
# 2. Überprüfen und ggf. Koordinatensysteme angleichen
with rasterio.open(nir_mc) as src:
    raster_crs = src.crs

if districts_mc.crs != raster_crs:
    districts_mc = districts_mc.to_crs(raster_crs)

In [15]:
def calculate_ndvi(nir, red):
    # NDVI-Formel: (NIR - RED) / (NIR + RED)
    ndvi = (nir_mc - red_mc) / (nir_mc + red_mc)
    return np.clip(ndvi, -1, 1)  # Werte zwischen -1 und 1 begrenzen

# Funktion, um Rasterdaten auf eine Geometrie zu beschränken
def clip_raster_to_geometry(raster_path, geometry):
    with rasterio.open(raster_path) as src:
        out_image, out_transform = mask(src, [geometry], crop=True)
        return out_image[0]  # Nur das erste Band zurückgeben

In [34]:
# 4. NDVI pro Alcaldía berechnen
ndvi_results = []

for _, alcaldia in districts_mc.iterrows():
    geometry = districts_mc.geometry  # Geometrie der aktuellen Alcaldía
    try:
    # Rasterdaten zuschneiden
        nir = clip_raster_to_geometry(nir_mc, geometry)
        red = clip_raster_to_geometry(red_mc, geometry)
    
        # NDVI berechnen
        ndvi = calculate_ndvi(nir, red)
    
        # Mittelwert des NDVI
        mean_ndvi = np.nanmean(ndvi)
    
        # Ergebnis speichern
        ndvi_results.append({
            "alcaldia": districts["alc"],  # Passe den Spaltennamen an
            "mean_ndvi": mean_ndvi
        })
    except Exception as e:
        print(f"Fehler bei {districts_mc['alc']}: {e}")
        ndvi_results.append({
            "alcaldia": districts_mc["alc"],
            "mean_ndvi": None
        })

Fehler bei 0              Azcapotzalco
1              Azcapotzalco
2              Azcapotzalco
3              Azcapotzalco
4              Azcapotzalco
               ...         
1538    Venustiano Carranza
1539    Venustiano Carranza
1540    Venustiano Carranza
1541    Venustiano Carranza
1542    Venustiano Carranza
Name: alc, Length: 1543, dtype: object: Input shapes do not overlap raster.
Fehler bei 0              Azcapotzalco
1              Azcapotzalco
2              Azcapotzalco
3              Azcapotzalco
4              Azcapotzalco
               ...         
1538    Venustiano Carranza
1539    Venustiano Carranza
1540    Venustiano Carranza
1541    Venustiano Carranza
1542    Venustiano Carranza
Name: alc, Length: 1543, dtype: object: Input shapes do not overlap raster.
Fehler bei 0              Azcapotzalco
1              Azcapotzalco
2              Azcapotzalco
3              Azcapotzalco
4              Azcapotzalco
               ...         
1538    Venustiano Carranza
153